# Yeast Colony Growth Simulation: Normal vs Snowflake Yeast

This notebook simulates and compares two fundamentally different yeast growth paradigms using **vector-based cellular models**:

1. **Normal Yeast** (*Saccharomyces cerevisiae* wild-type): Cells divide and separate, forming dispersed colonies as individual cells diffuse through the substrate.

2. **Snowflake Yeast** (*S. cerevisiae* ACE2 knockout): Cells divide but remain permanently attached via chitinous bud scars at their **poles** (ends), creating fractal-like branched tree structures through **polar budding**. This represents an experimental model for the evolution of multicellularity.

Both models render cells as ellipses with realistic aspect ratios and orientations.

## Background

Snowflake yeast were created in the Ratcliff lab (Georgia Tech) by knocking out the ACE2 transcription factor, which normally activates genes responsible for degrading the cell wall connection between mother and daughter cells after budding. The resulting "multicellular" clusters undergo:
- Settling selection (larger clusters settle faster)
- Apoptotic division (branches break off to reproduce)
- Evolution of increased cell elongation (aspect ratio 1.2 → 2.7 over ~3000 generations)

**Key feature**: Snowflake yeast exhibit **polar budding**—daughters attach at the mother's poles (ends of the long axis), creating end-to-end chains that branch into tree structures.

In [1]:
# Core imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.patches import Ellipse, Circle
from matplotlib.collections import PatchCollection, EllipseCollection
from IPython.display import HTML, display
from dataclasses import dataclass, field
from typing import List, Tuple, Optional, Dict
import random
from collections import deque

# For reproducibility
np.random.seed(42)
random.seed(42)

# Animation settings
plt.rcParams['animation.embed_limit'] = 50  # MB limit for embedded animations

---

## Part 1: Normal Yeast Colony Simulation

Normal yeast colonies grow through a combination of:
- **Cell division**: Mother cells bud to produce daughter cells
- **Cell separation**: Daughters detach and become independent
- **Random movement**: Free cells undergo Brownian motion
- **Crowding effects**: Cells compete for space

We model this using the same vector-based approach as snowflake yeast, but with cells that **separate after division** and move independently.

In [ ]:
@dataclass
class NormalYeastCell:
    """Represents a single normal yeast cell (vector-based)."""
    position: np.ndarray           # 2D position
    radius: float                  # Cell radius
    orientation: float             # Angle in radians
    aspect_ratio: float            # Length/width (typically ~1.2 for yeast)
    age: int = 0                   # Division cycles since birth
    buds_produced: int = 0         # Number of daughters produced
    max_buds: int = 6              # Maximum daughter cells
    cell_id: int = 0
    
    def can_divide(self) -> bool:
        return self.buds_produced < self.max_buds


@dataclass
class NormalYeastParams:
    """Parameters for normal yeast colony simulation."""
    # Cell properties
    initial_radius: float = 3.5        # Cell radius
    aspect_ratio: float = 1.15         # Slightly elongated
    initial_cells: int = 3             # Starting colony size
    
    # Division
    division_prob: float = 0.4         # Per-cell division probability
    bud_size_ratio: float = 0.85       # Daughter size relative to mother
    
    # Movement (cells separate and diffuse)
    separation_force: float = 2.0      # How fast cells push apart after division
    diffusion_strength: float = 0.3    # Random Brownian motion
    repulsion_strength: float = 1.5    # Collision avoidance
    
    # Simulation
    max_cells: int = 300               # Maximum colony size
    max_steps: int = 80                # Simulation duration
    

class NormalYeastColony:
    """
    Vector-based simulation of normal (wild-type) yeast colony growth.
    
    Unlike snowflake yeast, daughter cells SEPARATE from mothers after
    division and can move independently, forming dispersed colonies.
    """
    
    def __init__(self, params: NormalYeastParams = None):
        self.params = params or NormalYeastParams()
        self.cells: List[NormalYeastCell] = []
        self.next_id = 0
        self.step_count = 0
        self.history: List[List[NormalYeastCell]] = []
        
        # Initialize colony
        self._initialize_colony()
    
    def _initialize_colony(self):
        """Create initial cells at origin."""
        for i in range(self.params.initial_cells):
            angle = 2 * np.pi * i / self.params.initial_cells
            offset = self.params.initial_radius * 0.5
            pos = np.array([np.cos(angle) * offset, np.sin(angle) * offset])
            
            cell = NormalYeastCell(
                position=pos,
                radius=self.params.initial_radius,
                orientation=np.random.uniform(0, 2*np.pi),
                aspect_ratio=self.params.aspect_ratio,
                cell_id=self.next_id
            )
            self.cells.append(cell)
            self.next_id += 1
    
    def _get_bud_position(self, mother: NormalYeastCell) -> Tuple[np.ndarray, float]:
        """
        Calculate daughter position - buds from random location on mother surface.
        """
        # Random budding angle (can bud anywhere around the cell)
        bud_angle = np.random.uniform(0, 2 * np.pi)
        
        # Distance from mother center to daughter center
        daughter_radius = mother.radius * self.params.bud_size_ratio
        separation = mother.radius + daughter_radius * 1.1
        
        # Daughter position
        direction = np.array([np.cos(bud_angle), np.sin(bud_angle)])
        position = mother.position + direction * separation
        
        # Daughter orientation (random, independent of mother)
        orientation = np.random.uniform(0, 2*np.pi)
        
        return position, orientation
    
    def _check_collision(self, pos: np.ndarray, radius: float, 
                         exclude_id: Optional[int] = None) -> bool:
        """Check if position would collide with existing cells."""
        for cell in self.cells:
            if cell.cell_id == exclude_id:
                continue
            dist = np.linalg.norm(pos - cell.position)
            min_dist = (cell.radius + radius) * 0.85
            if dist < min_dist:
                return True
        return False
    
    def _apply_physics(self):
        """Apply separation forces and random diffusion to all cells."""
        forces = {cell.cell_id: np.zeros(2) for cell in self.cells}
        
        # Calculate repulsion forces between overlapping cells
        for i, cell1 in enumerate(self.cells):
            for cell2 in self.cells[i+1:]:
                diff = cell1.position - cell2.position
                dist = np.linalg.norm(diff)
                min_dist = (cell1.radius + cell2.radius) * 1.0
                
                if dist < min_dist and dist > 0.01:
                    # Repulsion force
                    overlap = min_dist - dist
                    force_mag = overlap * self.params.repulsion_strength
                    force_dir = diff / dist
                    
                    forces[cell1.cell_id] += force_dir * force_mag
                    forces[cell2.cell_id] -= force_dir * force_mag
        
        # Apply forces and random diffusion
        for cell in self.cells:
            # Apply repulsion
            cell.position += forces[cell.cell_id]
            
            # Random Brownian motion
            cell.position += np.random.normal(0, self.params.diffusion_strength, 2)
            
            # Slight random rotation
            cell.orientation += np.random.normal(0, 0.05)
    
    def step(self) -> int:
        """Execute one simulation step. Returns number of new cells."""
        if len(self.cells) >= self.params.max_cells:
            return 0
        
        new_cells = []
        
        for cell in self.cells:
            cell.age += 1
            
            # Division check
            if not cell.can_divide():
                continue
            if np.random.random() > self.params.division_prob:
                continue
            if len(self.cells) + len(new_cells) >= self.params.max_cells:
                break
            
            # Attempt division
            for attempt in range(5):
                bud_pos, bud_orient = self._get_bud_position(cell)
                bud_radius = cell.radius * self.params.bud_size_ratio
                
                if not self._check_collision(bud_pos, bud_radius, cell.cell_id):
                    daughter = NormalYeastCell(
                        position=bud_pos,
                        radius=bud_radius,
                        orientation=bud_orient,
                        aspect_ratio=self.params.aspect_ratio,
                        cell_id=self.next_id
                    )
                    new_cells.append(daughter)
                    cell.buds_produced += 1
                    self.next_id += 1
                    break
        
        self.cells.extend(new_cells)
        
        # Apply physics (separation and diffusion)
        self._apply_physics()
        
        self.step_count += 1
        return len(new_cells)
    
    def run(self) -> List[List[NormalYeastCell]]:
        """Run simulation, saving snapshots at each step."""
        self.history = [self._copy_cells()]
        
        for _ in range(self.params.max_steps):
            self.step()
            self.history.append(self._copy_cells())
        
        return self.history
    
    def _copy_cells(self) -> List[NormalYeastCell]:
        """Create deep copy of current cell list."""
        return [
            NormalYeastCell(
                position=c.position.copy(),
                radius=c.radius,
                orientation=c.orientation,
                aspect_ratio=c.aspect_ratio,
                age=c.age,
                buds_produced=c.buds_produced,
                max_buds=c.max_buds,
                cell_id=c.cell_id
            )
            for c in self.cells
        ]
    
    def get_cell_count(self) -> int:
        return len(self.cells)

In [ ]:
# Run normal yeast simulation
print("Running normal yeast colony simulation...")
normal_params = NormalYeastParams(
    initial_radius=3.5,
    aspect_ratio=1.15,
    initial_cells=3,
    division_prob=0.45,
    max_cells=250,
    max_steps=60,
    diffusion_strength=0.4,
    repulsion_strength=1.5
)

normal_colony = NormalYeastColony(normal_params)
normal_history = normal_colony.run()

print(f"Simulation complete: {len(normal_history)} frames")
print(f"Final cell count: {normal_colony.get_cell_count()}")

In [ ]:
def create_normal_yeast_animation(history: List[List[NormalYeastCell]], 
                                   interval: int = 100) -> animation.FuncAnimation:
    """
    Create animated visualization of normal yeast colony growth.
    Cells rendered as ellipses (vector-based, matching snowflake style).
    """
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Compute bounds from final frame
    final_cells = history[-1]
    if final_cells:
        all_x = [c.position[0] for c in final_cells]
        all_y = [c.position[1] for c in final_cells]
        max_r = max(c.radius for c in final_cells)
        margin = max_r * 3
        xlim = (min(all_x) - margin, max(all_x) + margin)
        ylim = (min(all_y) - margin, max(all_y) + margin)
        max_range = max(xlim[1] - xlim[0], ylim[1] - ylim[0]) / 2
        cx = (xlim[0] + xlim[1]) / 2
        cy = (ylim[0] + ylim[1]) / 2
    else:
        cx, cy, max_range = 0, 0, 50
    
    ax.set_xlim(cx - max_range, cx + max_range)
    ax.set_ylim(cy - max_range, cy + max_range)
    ax.set_aspect('equal')
    ax.set_title('Normal Yeast Colony Growth', fontsize=14)
    ax.set_xlabel('X position (μm)')
    ax.set_ylabel('Y position (μm)')
    ax.set_facecolor('#f5f5dc')  # Beige background for normal yeast
    
    # Colormap - warm colors for normal yeast
    cmap = plt.cm.YlOrBr
    
    def update(frame):
        ax.clear()
        cells = history[frame]
        
        if not cells:
            return []
        
        ax.set_xlim(cx - max_range, cx + max_range)
        ax.set_ylim(cy - max_range, cy + max_range)
        ax.set_aspect('equal')
        ax.set_facecolor('#f5f5dc')
        ax.set_title('Normal Yeast Colony Growth', fontsize=14)
        
        max_age = max(c.age for c in cells) if cells else 1
        
        # Draw cells as ellipses
        for cell in cells:
            color = cmap(0.3 + 0.5 * cell.age / max(max_age, 1))
            
            # Ellipse dimensions (width along orientation, height perpendicular)
            width = cell.radius * 2 * cell.aspect_ratio
            height = cell.radius * 2
            angle = np.degrees(cell.orientation)
            
            ellipse = Ellipse(
                xy=cell.position,
                width=width,
                height=height,
                angle=angle,
                facecolor=color,
                edgecolor='#8B4513',
                linewidth=0.5,
                alpha=0.85,
                zorder=2
            )
            ax.add_patch(ellipse)
        
        # Text annotation
        ax.text(0.02, 0.98, f'Frame: {frame}\nCells: {len(cells)}',
                transform=ax.transAxes, fontsize=12, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        return []
    
    anim = animation.FuncAnimation(fig, update, frames=len(history),
                                   interval=interval, blit=False)
    plt.close(fig)
    return anim

# Create and display animation
print("Creating normal yeast animation...")
normal_anim = create_normal_yeast_animation(normal_history, interval=100)
display(HTML(normal_anim.to_jshtml()))

---

## Part 2: Snowflake Yeast Simulation

Snowflake yeast (*S. cerevisiae* with ACE2 knockout) exhibit fundamentally different growth:

1. **Incomplete cytokinesis**: Daughter cells remain attached to mothers
2. **Branching tree topology**: The cluster forms a fractal-like tree structure
3. **No cell repositioning**: Once divided, cells cannot move relative to each other
4. **Cluster-level reproduction**: Large clusters break at weak points

We implement this as an L-system-inspired model where each cell tracks its parent connection, visualized as a 2D projection of the growing cluster.

In [ ]:
@dataclass
class SnowflakeYeastCell:
    """Represents a single cell in the snowflake yeast cluster."""
    position: np.ndarray           # 2D position
    radius: float                  # Cell radius
    orientation: float             # Angle in radians (long axis direction)
    aspect_ratio: float            # Length/width (1.0 = circular)
    generation: int                # Distance from founder
    age: int = 0                   # Division cycles since birth
    buds_produced: int = 0         # Number of daughters
    max_buds: int = 4              # Maximum daughter cells
    parent_id: Optional[int] = None
    cell_id: int = 0
    
    def can_divide(self) -> bool:
        return self.buds_produced < self.max_buds
    
    def get_pole_position(self, which_end: int = 1) -> np.ndarray:
        """
        Get position of cell pole (end of ellipse along long axis).
        which_end: +1 for forward pole, -1 for backward pole
        """
        # Semi-major axis length (half the long dimension)
        semi_major = self.radius * self.aspect_ratio
        direction = np.array([np.cos(self.orientation), np.sin(self.orientation)])
        return self.position + which_end * semi_major * direction


@dataclass 
class SnowflakeYeastParams:
    """Parameters for snowflake yeast simulation."""
    # Cell properties
    initial_radius: float = 3.0        # Cell radius (arbitrary units)
    aspect_ratio: float = 1.2          # Ancestral aspect ratio
    evolved_aspect_ratio: float = 2.7  # Evolved (macroscopic) aspect ratio
    use_evolved: bool = False          # Use evolved morphology
    
    # Polar budding geometry (end-to-end attachment)
    bud_angle_deviation: float = 15.0  # Max deviation from polar axis (degrees)
    bud_size_ratio: float = 0.8        # Daughter size relative to mother
    orientation_noise: float = 10.0    # Daughter orientation deviation (degrees)
    
    # Growth dynamics  
    division_prob: float = 0.6         # Per-cell division probability
    max_generations: int = 8           # Maximum tree depth
    max_cells: int = 300               # Maximum cluster size
    max_steps: int = 30                # Simulation duration
    
    # Collision
    collision_detection: bool = True
    min_separation: float = 0.9        # Fraction of sum of radii


class SnowflakeYeastCluster:
    """
    Simulation of snowflake yeast cluster growth.
    
    Daughter cells remain permanently attached at the POLES (ends) of
    mother cells, creating end-to-end chains that branch into tree structures.
    This is called "polar budding" - buds emerge from cell poles.
    """
    
    def __init__(self, params: SnowflakeYeastParams = None):
        self.params = params or SnowflakeYeastParams()
        self.cells: List[SnowflakeYeastCell] = []
        self.next_id = 0
        self.step_count = 0
        self.history: List[List[SnowflakeYeastCell]] = []
        
        # Track which poles are used for each cell
        self.used_poles: Dict[int, List[int]] = {}  # cell_id -> list of used pole directions
        
        # Initialize with founder cell
        self._create_founder()
    
    def _create_founder(self):
        """Create the founding cell at origin."""
        aspect = (self.params.evolved_aspect_ratio if self.params.use_evolved 
                  else self.params.aspect_ratio)
        
        founder = SnowflakeYeastCell(
            position=np.array([0.0, 0.0]),
            radius=self.params.initial_radius,
            orientation=np.random.uniform(0, 2*np.pi),
            aspect_ratio=aspect,
            generation=0,
            cell_id=self.next_id
        )
        self.cells.append(founder)
        self.used_poles[self.next_id] = []
        self.next_id += 1
    
    def _get_bud_position(self, mother: SnowflakeYeastCell) -> Tuple[np.ndarray, float, int]:
        """
        Calculate daughter position using POLAR BUDDING (end-to-end).
        
        Daughters bud from the poles (ends) of the mother's long axis,
        creating aligned chains that branch into tree structures.
        
        Returns: (position, orientation, pole_used)
        """
        aspect = (self.params.evolved_aspect_ratio if self.params.use_evolved 
                  else self.params.aspect_ratio)
        
        # Determine which pole to use
        used = self.used_poles.get(mother.cell_id, [])
        available_poles = [p for p in [1, -1] if p not in used]
        
        if not available_poles:
            # Both poles used, pick randomly for additional buds (side branching)
            pole = np.random.choice([1, -1])
            # Add slight lateral offset for side buds
            lateral_offset = np.random.uniform(-0.3, 0.3)
        else:
            pole = np.random.choice(available_poles)
            lateral_offset = 0
        
        # Base direction is along mother's long axis
        base_angle = mother.orientation + (0 if pole == 1 else np.pi)
        
        # Add small angular deviation for branching variety
        angle_dev = np.radians(np.random.uniform(
            -self.params.bud_angle_deviation, 
            self.params.bud_angle_deviation
        ))
        bud_angle = base_angle + angle_dev + lateral_offset
        
        # Calculate positions at cell poles (ends of ellipses)
        mother_semi_major = mother.radius * mother.aspect_ratio
        daughter_radius = mother.radius * self.params.bud_size_ratio
        daughter_semi_major = daughter_radius * aspect
        
        # Direction from mother center toward bud
        bud_direction = np.array([np.cos(bud_angle), np.sin(bud_angle)])
        
        # Daughter center: mother's pole + daughter's semi-major (end-to-end contact)
        separation = mother_semi_major + daughter_semi_major * 0.95  # Slight overlap at bud scar
        daughter_position = mother.position + bud_direction * separation
        
        # Daughter orientation continues the chain (aligned with mother)
        # Small deviation creates gradual curving of branches
        orientation_noise = np.radians(np.random.uniform(
            -self.params.orientation_noise,
            self.params.orientation_noise
        ))
        daughter_orientation = bud_angle + orientation_noise
        
        return daughter_position, daughter_orientation, pole
    
    def _check_collision(self, new_pos: np.ndarray, new_radius: float, 
                         exclude_id: Optional[int] = None) -> bool:
        """Check if a new cell would collide with existing cells."""
        if not self.params.collision_detection:
            return False
        
        aspect = (self.params.evolved_aspect_ratio if self.params.use_evolved 
                  else self.params.aspect_ratio)
        new_semi_major = new_radius * aspect
        
        for cell in self.cells:
            if cell.cell_id == exclude_id:
                continue
            
            dist = np.linalg.norm(new_pos - cell.position)
            # Use average of semi-major axes for collision check
            cell_semi_major = cell.radius * cell.aspect_ratio
            min_dist = (cell_semi_major + new_semi_major) * self.params.min_separation
            
            if dist < min_dist:
                return True
        
        return False
    
    def step(self) -> int:
        """Execute one division cycle. Returns number of new cells created."""
        if len(self.cells) >= self.params.max_cells:
            return 0
        
        new_cells = []
        
        for cell in self.cells:
            cell.age += 1
            
            # Skip if cannot divide
            if not cell.can_divide():
                continue
            if cell.generation >= self.params.max_generations:
                continue
            if np.random.random() > self.params.division_prob:
                continue
            if len(self.cells) + len(new_cells) >= self.params.max_cells:
                break
            
            # Attempt division (try a few times if collision)
            for attempt in range(5):
                bud_pos, bud_orient, pole_used = self._get_bud_position(cell)
                bud_radius = cell.radius * self.params.bud_size_ratio
                
                if not self._check_collision(bud_pos, bud_radius, cell.cell_id):
                    aspect = (self.params.evolved_aspect_ratio if self.params.use_evolved
                              else self.params.aspect_ratio)
                    
                    daughter = SnowflakeYeastCell(
                        position=bud_pos,
                        radius=bud_radius,
                        orientation=bud_orient,
                        aspect_ratio=aspect,
                        generation=cell.generation + 1,
                        parent_id=cell.cell_id,
                        cell_id=self.next_id
                    )
                    new_cells.append(daughter)
                    cell.buds_produced += 1
                    
                    # Track which pole was used
                    if cell.cell_id not in self.used_poles:
                        self.used_poles[cell.cell_id] = []
                    if pole_used not in self.used_poles[cell.cell_id]:
                        self.used_poles[cell.cell_id].append(pole_used)
                    
                    self.used_poles[self.next_id] = [-pole_used]  # Mark parent-facing pole as used
                    self.next_id += 1
                    break
        
        self.cells.extend(new_cells)
        self.step_count += 1
        return len(new_cells)
    
    def run(self) -> List[List[SnowflakeYeastCell]]:
        """Run simulation, saving snapshots at each step."""
        self.history = [self._copy_cells()]
        
        for _ in range(self.params.max_steps):
            added = self.step()
            self.history.append(self._copy_cells())
            
            if added == 0 and len(self.cells) > 5:
                for _ in range(3):
                    self.history.append(self._copy_cells())
                break
        
        return self.history
    
    def _copy_cells(self) -> List[SnowflakeYeastCell]:
        """Create deep copy of current cell list."""
        return [
            SnowflakeYeastCell(
                position=c.position.copy(),
                radius=c.radius,
                orientation=c.orientation,
                aspect_ratio=c.aspect_ratio,
                generation=c.generation,
                age=c.age,
                buds_produced=c.buds_produced,
                max_buds=c.max_buds,
                parent_id=c.parent_id,
                cell_id=c.cell_id
            )
            for c in self.cells
        ]
    
    def get_connectivity(self) -> List[Tuple[int, int]]:
        """Get parent-child edges for tree visualization."""
        edges = []
        for cell in self.cells:
            if cell.parent_id is not None:
                edges.append((cell.parent_id, cell.cell_id))
        return edges

In [ ]:
# Run snowflake yeast simulation (ancestral morphology)
print("Running snowflake yeast simulation (ancestral)...")
snowflake_params = SnowflakeYeastParams(
    initial_radius=4.0,
    aspect_ratio=1.2,              # Ancestral: nearly circular
    use_evolved=False,
    division_prob=0.65,
    max_generations=7,
    max_cells=200,
    max_steps=25,
    bud_angle_deviation=20.0,      # Small deviation from polar axis
    orientation_noise=15.0,        # Slight orientation variation
    bud_size_ratio=0.85
)

snowflake_cluster = SnowflakeYeastCluster(snowflake_params)
snowflake_history = snowflake_cluster.run()

print(f"Simulation complete: {len(snowflake_history)} frames")
print(f"Final cell count: {len(snowflake_cluster.cells)}")
print(f"Max generation: {max(c.generation for c in snowflake_cluster.cells)}")

In [ ]:
def create_snowflake_animation(history: List[List[SnowflakeYeastCell]], 
                                interval: int = 200,
                                show_connections: bool = True) -> animation.FuncAnimation:
    """
    Create animated visualization of snowflake yeast cluster growth.
    Cells are rendered as ellipses aligned end-to-end (polar budding).
    """
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Compute bounds from final frame
    final_cells = history[-1]
    if final_cells:
        all_x = [c.position[0] for c in final_cells]
        all_y = [c.position[1] for c in final_cells]
        max_r = max(c.radius * c.aspect_ratio for c in final_cells)
        margin = max_r * 3
        xlim = (min(all_x) - margin, max(all_x) + margin)
        ylim = (min(all_y) - margin, max(all_y) + margin)
        # Make square
        max_range = max(xlim[1] - xlim[0], ylim[1] - ylim[0]) / 2
        cx = (xlim[0] + xlim[1]) / 2
        cy = (ylim[0] + ylim[1]) / 2
        ax.set_xlim(cx - max_range, cx + max_range)
        ax.set_ylim(cy - max_range, cy + max_range)
    else:
        cx, cy, max_range = 0, 0, 50
        ax.set_xlim(-50, 50)
        ax.set_ylim(-50, 50)
    
    ax.set_aspect('equal')
    ax.set_title('Snowflake Yeast Cluster Growth (Polar Budding)', fontsize=14)
    ax.set_xlabel('X position (μm)')
    ax.set_ylabel('Y position (μm)')
    ax.set_facecolor('#f0f0f0')
    
    # Colormap for generations
    cmap = plt.cm.viridis
    
    def update(frame):
        ax.clear()
        
        cells = history[frame]
        if not cells:
            return []
        
        # Restore axis settings
        if final_cells:
            ax.set_xlim(cx - max_range, cx + max_range)
            ax.set_ylim(cy - max_range, cy + max_range)
        ax.set_aspect('equal')
        ax.set_facecolor('#f0f0f0')
        ax.set_title('Snowflake Yeast Cluster Growth (Polar Budding)', fontsize=14)
        
        max_gen = max(c.generation for c in cells)
        
        # Draw connections first (behind cells)
        if show_connections:
            cell_dict = {c.cell_id: c for c in cells}
            for cell in cells:
                if cell.parent_id is not None and cell.parent_id in cell_dict:
                    parent = cell_dict[cell.parent_id]
                    ax.plot([parent.position[0], cell.position[0]],
                            [parent.position[1], cell.position[1]],
                            'k-', linewidth=0.5, alpha=0.3, zorder=1)
        
        # Draw cells as ellipses
        # For matplotlib Ellipse: width is horizontal, height is vertical BEFORE rotation
        # orientation rotates the ellipse so that the long axis points in that direction
        for cell in cells:
            color = cmap(cell.generation / max(max_gen, 1))
            
            # Ellipse dimensions:
            # - width (along orientation after rotation) = major axis = 2 * radius * aspect_ratio
            # - height (perpendicular) = minor axis = 2 * radius
            # matplotlib Ellipse: width is x-dimension, height is y-dimension before rotation
            # angle rotates counterclockwise
            # So we set width=major, height=minor, angle=orientation
            width = cell.radius * 2 * cell.aspect_ratio   # Long axis (major)
            height = cell.radius * 2                       # Short axis (minor)
            angle = np.degrees(cell.orientation)
            
            ellipse = Ellipse(
                xy=cell.position,
                width=width,
                height=height,
                angle=angle,
                facecolor=color,
                edgecolor='black',
                linewidth=0.5,
                alpha=0.8,
                zorder=2
            )
            ax.add_patch(ellipse)
        
        # Update text
        ax.text(0.02, 0.98, f'Frame: {frame}\nCells: {len(cells)}',
                transform=ax.transAxes, fontsize=12,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        return []
    
    anim = animation.FuncAnimation(fig, update, frames=len(history),
                                   interval=interval, blit=False)
    plt.close(fig)
    return anim

# Create and display animation
print("Creating snowflake yeast animation...")
snowflake_anim = create_snowflake_animation(snowflake_history, interval=300)
display(HTML(snowflake_anim.to_jshtml()))

---

## Part 3: Evolved Macroscopic Snowflake Yeast

After ~3,000 generations of selection for larger size (settling selection), snowflake yeast evolved:

1. **Elongated cells**: Aspect ratio increased from ~1.2 to ~2.7
2. **Larger clusters**: ~20,000× more cells per cluster
3. **Increased toughness**: Material properties changed from gelatin-like to wood-like
4. **Branch entanglement**: Branches wrap around each other, increasing structural integrity

Let's simulate the evolved morphology:

In [ ]:
# Run evolved snowflake yeast simulation
print("Running evolved snowflake yeast simulation...")
evolved_params = SnowflakeYeastParams(
    initial_radius=4.0,
    aspect_ratio=2.7,              # Evolved: elongated cells
    evolved_aspect_ratio=2.7,
    use_evolved=True,
    division_prob=0.7,
    max_generations=9,
    max_cells=250,
    max_steps=30,
    bud_angle_deviation=18.0,      # Slightly tighter polar budding
    orientation_noise=12.0,        # Less deviation = straighter chains
    bud_size_ratio=0.85,
    collision_detection=True,
    min_separation=0.85
)

evolved_cluster = SnowflakeYeastCluster(evolved_params)
evolved_history = evolved_cluster.run()

print(f"Simulation complete: {len(evolved_history)} frames")
print(f"Final cell count: {len(evolved_cluster.cells)}")
print(f"Max generation: {max(c.generation for c in evolved_cluster.cells)}")

In [9]:
# Create and display evolved animation
print("Creating evolved snowflake yeast animation...")
evolved_anim = create_snowflake_animation(evolved_history, interval=300)
display(HTML(evolved_anim.to_jshtml()))

Creating evolved snowflake yeast animation...


---

## Part 4: Side-by-Side Comparison

Let's compare all three growth modes:

In [ ]:
def plot_comparison(normal_cells: List[NormalYeastCell],
                    ancestral_cells: List[SnowflakeYeastCell],
                    evolved_cells: List[SnowflakeYeastCell]):
    """
    Create side-by-side comparison of final colony morphologies.
    All three now use vector-based (ellipse) rendering.
    """
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Helper function to render cells
    def render_cells(ax, cells, cmap, title, color_by='age', bg_color='#f5f5dc'):
        ax.set_aspect('equal')
        ax.set_facecolor(bg_color)
        
        if not cells:
            ax.set_title(title)
            return
        
        all_x = [c.position[0] for c in cells]
        all_y = [c.position[1] for c in cells]
        max_r = max(c.radius * getattr(c, 'aspect_ratio', 1.0) for c in cells)
        margin = max_r * 2
        max_range = max(max(all_x) - min(all_x), max(all_y) - min(all_y)) / 2 + margin
        cx, cy = np.mean(all_x), np.mean(all_y)
        ax.set_xlim(cx - max_range, cx + max_range)
        ax.set_ylim(cy - max_range, cy + max_range)
        
        if color_by == 'age':
            max_val = max(c.age for c in cells) if cells else 1
            get_val = lambda c: c.age
        else:  # generation
            max_val = max(c.generation for c in cells) if cells else 1
            get_val = lambda c: c.generation
        
        for cell in cells:
            color = cmap(get_val(cell) / max(max_val, 1))
            aspect = getattr(cell, 'aspect_ratio', 1.0)
            width = cell.radius * 2 * aspect
            height = cell.radius * 2
            angle = np.degrees(cell.orientation)
            
            ellipse = Ellipse(
                xy=cell.position, width=width, height=height,
                angle=angle, facecolor=color, edgecolor='black',
                linewidth=0.3, alpha=0.8
            )
            ax.add_patch(ellipse)
        
        ax.set_title(title, fontsize=12)
    
    # Panel 1: Normal yeast
    render_cells(axes[0], normal_cells, plt.cm.YlOrBr,
                 f'Normal Yeast\n({len(normal_cells)} cells)',
                 color_by='age', bg_color='#f5f5dc')
    
    # Panel 2: Ancestral snowflake
    render_cells(axes[1], ancestral_cells, plt.cm.viridis,
                 f'Ancestral Snowflake Yeast\n({len(ancestral_cells)} cells, AR={ancestral_cells[0].aspect_ratio:.1f})',
                 color_by='generation', bg_color='#f0f0f0')
    
    # Panel 3: Evolved snowflake
    render_cells(axes[2], evolved_cells, plt.cm.plasma,
                 f'Evolved Snowflake Yeast\n({len(evolved_cells)} cells, AR={evolved_cells[0].aspect_ratio:.1f})',
                 color_by='generation', bg_color='#f0f0f0')
    
    plt.tight_layout()
    return fig

# Generate comparison plot
fig = plot_comparison(
    normal_history[-1],
    snowflake_history[-1],
    evolved_history[-1]
)
plt.show()

---

## Part 5: Quantitative Analysis

In [11]:
def analyze_cluster(cells: List[SnowflakeYeastCell], name: str) -> Dict:
    """
    Compute morphological metrics for snowflake yeast cluster.
    """
    if len(cells) < 2:
        return {'name': name, 'n_cells': len(cells)}
    
    positions = np.array([c.position for c in cells])
    
    # Centroid and distances
    centroid = positions.mean(axis=0)
    distances = np.linalg.norm(positions - centroid, axis=1)
    
    # Radius of gyration
    Rg = np.sqrt(np.mean(distances**2))
    
    # Fractal dimension estimate (mass-radius scaling)
    r_bins = np.linspace(distances.min() + 0.1, distances.max(), 15)
    N_within = [np.sum(distances <= r) for r in r_bins]
    
    valid = (np.array(N_within) > 0) & (r_bins > 0)
    if np.sum(valid) > 3:
        log_r = np.log(r_bins[valid])
        log_N = np.log(np.array(N_within)[valid])
        D_fractal = np.polyfit(log_r, log_N, 1)[0]
    else:
        D_fractal = np.nan
    
    # Generation statistics
    generations = [c.generation for c in cells]
    max_gen = max(generations)
    
    # Branching ratio
    parent_counts = {}
    for cell in cells:
        if cell.parent_id is not None:
            parent_counts[cell.parent_id] = parent_counts.get(cell.parent_id, 0) + 1
    avg_branching = np.mean(list(parent_counts.values())) if parent_counts else 1.0
    
    return {
        'name': name,
        'n_cells': len(cells),
        'cluster_radius': distances.max(),
        'radius_of_gyration': Rg,
        'fractal_dimension': D_fractal,
        'max_generation': max_gen,
        'avg_branching_ratio': avg_branching,
        'aspect_ratio': cells[0].aspect_ratio
    }

# Analyze both snowflake variants
ancestral_metrics = analyze_cluster(snowflake_history[-1], 'Ancestral')
evolved_metrics = analyze_cluster(evolved_history[-1], 'Evolved')

# Print comparison table
print("\n" + "="*60)
print("MORPHOLOGICAL COMPARISON: Ancestral vs Evolved Snowflake Yeast")
print("="*60)
print(f"{'Metric':<25} {'Ancestral':>15} {'Evolved':>15}")
print("-"*60)

for key in ['n_cells', 'cluster_radius', 'radius_of_gyration', 
            'fractal_dimension', 'max_generation', 'avg_branching_ratio', 'aspect_ratio']:
    if key in ancestral_metrics:
        val1 = ancestral_metrics[key]
        val2 = evolved_metrics[key]
        if isinstance(val1, float):
            print(f"{key:<25} {val1:>15.2f} {val2:>15.2f}")
        else:
            print(f"{key:<25} {val1:>15} {val2:>15}")
print("="*60)


MORPHOLOGICAL COMPARISON: Ancestral vs Evolved Snowflake Yeast
Metric                          Ancestral         Evolved
------------------------------------------------------------
n_cells                               200             250
cluster_radius                      25.16           24.64
radius_of_gyration                  18.31           18.09
fractal_dimension                    2.62            3.04
max_generation                          7               9
avg_branching_ratio                  1.75            1.72
aspect_ratio                         1.20            2.70


In [ ]:
# Plot growth curves
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Left: Cell count over time
ax1 = axes[0]
normal_counts = [len(frame) for frame in normal_history]
ancestral_counts = [len(frame) for frame in snowflake_history]
evolved_counts = [len(frame) for frame in evolved_history]

ax1.plot(normal_counts, 'o-', label='Normal Yeast', color='#D2691E', markersize=4)
ax1.plot(ancestral_counts, 's-', label='Ancestral Snowflake', color='#2E8B57', markersize=4)
ax1.plot(evolved_counts, '^-', label='Evolved Snowflake', color='#8B008B', markersize=4)

ax1.set_xlabel('Simulation Frame')
ax1.set_ylabel('Cell Count')
ax1.set_title('Colony Growth Over Time')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Generation distribution for snowflake variants
ax2 = axes[1]

ancestral_gens = [c.generation for c in snowflake_history[-1]]
evolved_gens = [c.generation for c in evolved_history[-1]]

max_gen = max(max(ancestral_gens), max(evolved_gens))
bins = np.arange(0, max_gen + 2) - 0.5

ax2.hist(ancestral_gens, bins=bins, alpha=0.6, label='Ancestral', color='#2E8B57')
ax2.hist(evolved_gens, bins=bins, alpha=0.6, label='Evolved', color='#8B008B')

ax2.set_xlabel('Generation (distance from founder)')
ax2.set_ylabel('Number of Cells')
ax2.set_title('Cell Distribution by Generation')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Summary

This notebook demonstrated three distinct yeast growth paradigms using **vector-based cellular simulations**:

### Normal Yeast (Wild-type)
- Cells divide and **separate** after budding
- Daughter cells can bud in any direction (random budding angles)
- Cells undergo **Brownian diffusion** and move independently
- Forms **dispersed, spreading colonies**

### Ancestral Snowflake Yeast (ACE2 knockout)
- Daughter cells **remain attached** at cell poles (polar budding)
- **End-to-end** attachment creates aligned chains
- Nearly circular cells (aspect ratio ~1.2)
- Chains branch to form **tree-like fractal structures**

### Evolved Macroscopic Snowflake Yeast
- ~3,000 generations of settling selection
- **Highly elongated cells** (aspect ratio ~2.7)
- Polar budding creates **straighter, longer chains**
- End-to-end alignment enables greater cluster size

### Key Biological Insight: Polar Budding

The transition from normal yeast to snowflake yeast involves:
1. **Incomplete cytokinesis**: ACE2 knockout prevents cell separation
2. **Polar budding**: Buds emerge from cell poles (ends), not randomly
3. **End-to-end attachment**: Cells align lengthwise like chains
4. **Branching**: When both poles are used, branches form at cell tips

This creates the characteristic "snowflake" morphology—a fractal tree of end-to-end connected cells.

### References

- Ratcliff, W.C., et al. (2012). "Experimental evolution of multicellularity." *PNAS*, 109(5), 1595-1600.
- Ratcliff, W.C., et al. (2015). "Origins of multicellular evolvability in snowflake yeast." *Nature Communications*, 6, 6102.
- Bozdag, G.O., et al. (2023). "De novo evolution of macroscopic multicellularity." *Nature*, 617, 747-754.